In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")

In [2]:
import requests
from datetime import date, datetime, timedelta
import time

In [3]:
# Testing geocoding to get the latlong data of a city
def get_city_coordinates(city, api_key):
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return lat, lon
        else:
            print(f"City '{city}' not found.")
            return None, None
    else:
        print("Error fetching coordinates:", response.status_code)
        return None, None

city = "Surabaya"
# city = "Jakarta"
lat, lon = get_city_coordinates(city, api_key)
print(lat)
print(lon)

-7.2459717
112.7378266


In [4]:
# Function to fetch weather data
def get_weather(lat, lon, part, api_key):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude={part}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error:", response.status_code)
        return None

# Function to display weather data
def display_weather(data):
    if data:
        temp = data["current"]["temp"]
        weather = data["current"]["weather"][0]["description"]
        humidity = data["current"]["humidity"]
        wind_speed = data["current"]["wind_speed"]

        # print(f"Weather in {city}:")
        print(f"Temperature: {temp}°C")
        print(f"Condition: {weather}")
        print(f"Humidity: {humidity}%")
        print(f"Wind Speed: {wind_speed} m/s")
    else:
        print("No data to display.")

# Main program
if __name__ == "__main__":
    city_name = input("Enter the city name: ")
    part = 'minutely,hourly'
    weather_data = get_weather(lat, lon, part, api_key)
    display_weather(weather_data)

Temperature: 29.25°C
Condition: overcast clouds
Humidity: 70%
Wind Speed: 5.41 m/s


In [30]:
def get_historical_weather(lat, lon, api_key, days=30):
    """Fetch historical weather data for the last 'days' days."""
    base_url = base_url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine"
    weather_data = []

    for i in range(days):
        # Calculate the timestamp for each day
        ts_d_min_i = datetime.utcnow() - timedelta(days=i+1) # Timestamp of day minus (i+1)
        timestamp = int(ts_d_min_i.timestamp())
        params = {
            "lat": lat,
            "lon": lon,
            "dt": timestamp,
            "appid": api_key,
            "units": "metric" # Optional: Metric units for temperature
        }

        # Make the API call
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            day_data = response.json()
            weather_data.append({
                "timestamp": timestamp,
                "date": datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d'),
                "temperature": day_data["data"][0]["temp"],
                "description": day_data["data"][0]["weather"][0]["description"]
            })
        else:
            print(f"Error fetching weather data for day {i+1}: {response.status_code}")
            break

        # Avoid hitting rate limits
        time.sleep(1)

    return weather_data

In [55]:
import csv

# Save the weather data to a CSV file
def save_to_csv(data, city_name):
    filename = "{}_weather.csv".format(city_name.lower())
    with open(filename, mode="w", newline="") as file:
        writer = csv.DictWriter(
            file,
            fieldnames=["timestamp", "date", "temperature", "description"])
        writer.writeheader()
        writer.writerows(data)
    print(f"Weather data saved to {filename}")


In [57]:
def main():
    # Change to your desired city
    city_name = "Surabaya"
    # city_name = input("Enter the city name: ")
    
    # Number of past days to retrieve
    days = 30

    # Step 1: Get city coordinates
    lat, lon = get_city_coordinates(city_name, api_key)
    if lat is None or lon is None:
        return

    # Step 2: Fetch historical weather data
    weather_data = get_historical_weather(lat, lon, api_key, days)

    # Step 3: Display or process the data
    for day in weather_data:
        timestamp = day["timestamp"]
        date = day["date"]
        temp = day["temperature"]
        weather = day["description"]
        print(f"Date: {date}, Time: {ts}, Temp: {temp}°C, Weather: {weather}")

    # Step 4: Store the weather data to a CSV file
    save_to_csv(weather_data, city_name)

if __name__ == "__main__":
    main()

Date: 2025-01-19, Time: 2025-01-20 06:25:45.740539, Temp: 27.79°C, Weather: haze
Date: 2025-01-18, Time: 2025-01-20 06:25:45.740539, Temp: 25.31°C, Weather: overcast clouds
Date: 2025-01-17, Time: 2025-01-20 06:25:45.740539, Temp: 26.2°C, Weather: overcast clouds
Date: 2025-01-16, Time: 2025-01-20 06:25:45.740539, Temp: 26.12°C, Weather: moderate rain
Date: 2025-01-15, Time: 2025-01-20 06:25:45.740539, Temp: 25.85°C, Weather: overcast clouds
Date: 2025-01-14, Time: 2025-01-20 06:25:45.740539, Temp: 26.24°C, Weather: overcast clouds
Date: 2025-01-13, Time: 2025-01-20 06:25:45.740539, Temp: 26.56°C, Weather: few clouds
Date: 2025-01-12, Time: 2025-01-20 06:25:45.740539, Temp: 26.47°C, Weather: few clouds
Date: 2025-01-11, Time: 2025-01-20 06:25:45.740539, Temp: 26.81°C, Weather: few clouds
Date: 2025-01-10, Time: 2025-01-20 06:25:45.740539, Temp: 26.14°C, Weather: few clouds
Date: 2025-01-09, Time: 2025-01-20 06:25:45.740539, Temp: 26.84°C, Weather: haze
Date: 2025-01-08, Time: 2025-01-2